# Assignment 1: Dataset Description and Exploration

(c) 2021 Tom Röschinger and Linlin Chen. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

***

In [1]:
from tdc.multi_pred import PPI
import pandas as pd
import numpy as np

import tensorflow as tf
import sklearn

import iqplot

#import panel as pn

import bokeh.io
import bokeh.plotting

#pn.extension()
bokeh.io.output_notebook()

Loading BokehJS ...

### Dataset Description

Source: https://tdcommons.ai/multi_pred_tasks/ppi/

Literature: https://www.nature.com/articles/s41586-020-2188-x#code-availability

The dataset is a reference map of the human protein interactome, with approximately 53,000 human binary protein interactions mapped using a yeast-2-hybrid assay. The dataset comes from a recent 2020 paper by Luck et al. entitled "A reference map of the human binary protein interactome". 



### Goal: 
Using this dataset, we hope to train a model that is able to predict whether or not pairs of amino acid sequences are able to interact in the context of this assay. We are also hoping to identify interaction motifs, the particular stretches of AAs that are directly involved in the interactions. 


Let's have a look at the data set!

In [13]:
data = PPI(name = 'HuRI')
data = data.neg_sample(frac=1)
split = data.get_split()
df = split['train']
#df.rename(columns={"Y" :"log IC50"}, inplace=True)
df.head()

Found local copy...
Loading...
Done!


,Protein1_ID,Protein1,Protein2_ID,Protein2,Y
0,ENSG00000000005,MAKNPPENCEDCHILNAEAFKSKKICKSLKICGLVFGILALTLIVL...,ENSG00000061656,MRRSSRPGSASSSRKHTPNFFSENSSMSITSEDSKGLRSAEPGPGE...,1
1,ENSG00000000005,MAKNPPENCEDCHILNAEAFKSKKICKSLKICGLVFGILALTLIVL...,ENSG00000104765,MSSHLVEPPPPLHNNNNNCEENEQSLPPPAGLNSSWVELPMNSSNG...,1
2,ENSG00000000005,MAKNPPENCEDCHILNAEAFKSKKICKSLKICGLVFGILALTLIVL...,ENSG00000105383,MPLLLLLPLLWAGALAMDPNFWLQVQESVTVQEGLCVLVPCTFFHP...,1
3,ENSG00000000005,MAKNPPENCEDCHILNAEAFKSKKICKSLKICGLVFGILALTLIVL...,ENSG00000124103,MWTLKSSLVLLLCLTCSYAFMFSSLRQKTSEPQGKVQYGEHFRIRQ...,1
4,ENSG00000000005,MAKNPPENCEDCHILNAEAFKSKKICKSLKICGLVFGILALTLIVL...,ENSG00000139637,MGHQFLRGLLTLLLPPPPLYTRHRMLGPESVPPPKRSRSKLMAPPR...,1


### Number of amino acids per sequence

In [3]:
protein_lengths = np.array([len(seq) for seq in df['Protein1'].unique()])

bokeh.io.show(
    iqplot.ecdf(data=protein_lengths, title="Number of Amino Acids per Sequence")
)

### Number of Proteins per sequence

In [4]:
protein_numbers = np.array([seq.count("*") for seq in df['Protein1'].unique()])

bokeh.io.show(
    iqplot.ecdf(data=protein_numbers, title='Number of Proteins per Sequence')
)

In [5]:
p = bokeh.plotting.figure(frame_width=400, frame_height=300)

p.scatter(protein_numbers, protein_lengths)

bokeh.io.show(p)

### One-hot encoding of Amino Acid Sequences

In [1]:
AA_dict = {
    '*':0,'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,
    'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20, 'X':21, 'U':22
}

In [ ]:
def one_hot_protein(seq_list):
    L = np.max([len(seq) for seq in seq_list])
    matrix_list = []
    
    for seq in seq_list:
        matrix = np.zeros([23, L])
        for i in range(len(seq)):
            matrix[AA_dict[seq[i]], i] = 1
        matrix_list.append(matrix)
    return matrix_list

df['Protein1_one_hot'] = one_hot_protein(df['Protein1'])
df['Protein2_one_hot'] = one_hot_protein(df['Protein2'])

In [ ]:
df.head()

## EDA

First, let's compute how many interactions each proteins has on average.

In [6]:
tested_interactions = df.groupby('Protein1_ID').agg(['mean', 'count']).reset_index()
tested_interactions.columns = tested_interactions.columns.droplevel()
tested_interactions.head()

,,mean,count
0,ENSG00000000005,0.764706,17
1,ENSG00000000419,0.250000,4
2,ENSG00000000457,0.571429,7
3,ENSG00000000460,0.250000,4
4,ENSG00000001036,0.562500,16


In [7]:
bokeh.io.show(
    iqplot.ecdf(data=tested_interactions, q="mean", x_axis_label="Interactions", title="Mean Number of positive interactions per Protein")
)

In [8]:
bokeh.io.show(
    iqplot.ecdf(data=tested_interactions, q="count", x_axis_label="Interactions", title="Mean Number of positive interactions per Protein")
)

In [10]:
p = bokeh.plotting.figure(frame_width=400, frame_height=300)

p.scatter(tested_interactions['count'], tested_interactions['mean'])

bokeh.io.show(p)

Now let's compute how the number of proteins per gene relates to the number of interactions.

In [11]:
def count_stars(seqs1, seqs2):
    interactions = np.zeros(len(seqs1))
    for i in range(len(seqs1)):
        interactions[i] = seqs1[i].count("*") * seqs2[i].count("*")
        
    return interactions

df['possible_interactions'] = count_stars(df['Protein1'], df['Protein2'])

total_interactions = df.groupby('possible_interactions').mean().reset_index()
total_interactions.head()

,possible_interactions,Y
0,0.0,0.000000
1,1.0,0.609924
2,2.0,0.528898
3,3.0,0.504374
4,4.0,0.503259


In [12]:
p = bokeh.plotting.figure()

p.scatter(total_interactions['possible_interactions'], total_interactions['Y'])

bokeh.io.show(p)

### Tensorflow

In [44]:
classes = df['Y'].values[0:1000]
p1 = df['Protein1_one_hot'].values[0:1000]
p2 = df['Protein2_one_hot'].values[0:1000]


Here we need to stack the array into a a single one with an extra dimension. However, if I try, the notebook dies :(

In [46]:
# This kills my notebook (probably too much memory needed)
np.stack(p1).shape

(1000, 23, 33472)

In [ ]:
flow_ob = tf.data.Dataset.from_tensor_slices((p1, p2_stack, classes))

In [ ]:
flow_ob.take(1)

## Computing Environment

In [ ]:
%load_ext watermark
%watermark -v -p tdc,pandas,numpy,tensorflow,sklearn,iqplot,panel,bokeh